Публичный ключ из приватного ключа вычиляется путём умножения на эллептических кривых `K = k * G`, где `K` - публичный ключ, `k` - приватный ключ, `G` - генераторная точка (base point), __является константой__. 

Сжатая форма:
```
G = 02 79BE667E F9DCBBAC 55A06295 CE870B07 029BFCDB 2DCE28D9 59F2815B 16F81798
```

Несжатая форма:
```
G = 04 79BE667E F9DCBBAC 55A06295 CE870B07 029BFCDB 2DCE28D9 59F2815B 16F81798 483ADA77 26A3C465 5DA4FBFC 0E1108A8 FD17B448 A6855419 9C47D08F FB10D4B8
```

`G` - это один из параметров эллиптической кривой `secp256k1`. Увидеть все параметры можно тут: http://www.secg.org/SEC2-Ver-1.0.pdf страница 15, параграф 2.7.1

Обратная операция умножения на эллептических кривых называется нахождением дискретного логарифма, вычисление `k` при известном `K` возможно лишь с помощью полного перебора.

Я не имею возможности углубиться в математику, поэтому не смогу продемонстрировать вычисления на более низком уровне. Будем использовать готовую библиотеку для работы с Bitcoin. Которая, в свою очередь, как и многие другие, для вычислений использует готовые решения из `openssl`.

Используем уже готовый приватный ключ, который я получил в предыдущем примере

In [58]:
priv_base58_key = '5HxLbpmHNe3gbzAktfpmKeU2TLdMwLQMxaiSVPEXsCXU6a7cE9y'

In [56]:
from bitcoin.wallet import CBitcoinSecret

private_key = CBitcoinSecret(priv_base58_key)

private_key.pub

CPubKey(b"\x04\x11\x08\xfe\xfcS\xc2UN\xb1Tls~f\xbe\x9b\xc4,\x9d{D\xc3\x116\xeeg\xc2fK\xfa\xcf\x9f`\x16\xdd\xe5\xc1'\xd8< \x8d\xec\x0b^\x9e\xba\x04\n\x12\xd9\xcfDf.\x81\x9cj4\xa9\x14\xe1~\xb3")

В случае использования такой высокоуровневой библиотеки как `python-bitcoinlib` для получения публичного ключа, нам стоит лишь обратиться к свойству `pub` объекта приватного ключа.

Теперь разберём процесс вычисления Bitcoin адреса из публичного ключа. Происходит он так: хешируем публичный ключ с помощью хеш-функции `sha256`, результат хешируем с помощью `ripemd160`, добавляем байт `0x00` в начало получившихся данных и кодируем это в `Base58Check`. 

В прошлый раз мы уже разбирали что представляют из себя 4 контрольных байта и как их получить. Повторим эту же процедуру и сейчас. Получим 4 байта, добавим в конец данных, результирующие данные кодируем в `Base58`, чтобы получить строку в `Base58Check`

In [59]:
import hashlib
import bitcoin.base58


ripemd160 = hashlib.new('ripemd160')

# ripemd160(sha256(public-key))
ripemd160.update(hashlib.sha256(private_key.pub).digest())

# checksum: два раза кодируем данные с помощью sha256 и берём первые 4 байта
check_sum_bytes = hashlib.sha256(hashlib.sha256(ripemd160.digest()).digest()).digest()[:4]

# 0x00 + ripemd160(sha256(public-key)) + check sum bytes
address_bytes = bytes([0]) + ripemd160.digest() + check_sum_bytes

address = bitcoin.base58.encode(address_bytes)

address

'1LcpkXnZVyknWWQMnoyRs6R6MBKJWU4Smj'

В реальных проектах, мы можем обойтись без написания лишнего кода, а использовать уже готовые решения

In [60]:
from bitcoin.wallet import CBitcoinAddress
from bitcoin.core.serialize import Hash160

CBitcoinAddress.from_bytes(Hash160(private_key.pub), 0)

P2PKHBitcoinAddress('1LcpkXnZVyknWWQMnoyRs6R6MBKJTrQFEv')

Получили тот же результат. Просто абстракции, которые предоставляет нам `python-bitcoinlib`, уместили это в одну строку. 

Функция `Hash160` просто делает `ripemd160(sha256(public-key))`, метод `from_bytes` класса `CBitcoinAddress` кодирует результат функции `Hash160` в `Base58Check` автоматически вычисляя контрольную сумму и подставляя в качестве префикса байт `0x00`, который мы переедали вторым аргументом.